This notebook will extract information from all of our DDOS csv files and place information into a large samples/features matrix. The matrix will look something like:
```
SOURCE_IP | VICTIM_IP | <Protocol>TRAFFIC_FROM_SRC_TO_DEST ...
```
Where `<Protocol>` can be ICMP, UDP, TCP, and a few others. 

In [195]:
import pandas as pd 
import numpy as np 
import csv as csv 
import glob 
import pickle
from datetime import datetime 
import matplotlib.pyplot as plt 
%matplotlib inline

In [196]:
victimIP = "71.126.222.64"
to_drop = ['<->', 'total_bytes', 'total_frames']
names = ['src', '<->', 'dest', 'in_frames', 'in_bytes', 'out_frames', 'out_bytes', 'total_frames', 'total_bytes', 'rel_start', 'duration']

In [197]:
# Begin dataframe by importing the total data, which will have all IPs
protocol_names = ['total' + "_" + name for name in names]
print(protocol_names)
protocol_to_drop = ['total' + "_" + name for name in to_drop]
ddos_df = pd.read_csv('resources/smallddos_total.tshark.csv', header=None, skiprows=5, names=protocol_names)
ddos_df = ddos_df.drop(ddos_df.tail(1).index)
ddos_df = ddos_df.drop(protocol_to_drop, axis=1)
ddos_df = ddos_df[ddos_df.total_src != victimIP]

['total_src', 'total_<->', 'total_dest', 'total_in_frames', 'total_in_bytes', 'total_out_frames', 'total_out_bytes', 'total_total_frames', 'total_total_bytes', 'total_rel_start', 'total_duration']


In [198]:
ddos_files = glob.glob('resources/smallddos*')
for i in range(len(ddos_files)):
    protocol = ddos_files[i][20:-11] # e.g., icmp, tcp, total, etc
    print("PROTOCOL : {0}".format(protocol))
    if protocol == 'total':
        pass
    else:
        # Prefix the column names with the name of the protocol, e.g.:
        #     icmp_src icmp_dest icmp_out_bytes ...
        protocol_names = [protocol + "_" + name for name in names]
        protocol_to_drop = [protocol + "_" + col for col in to_drop]
        
        # Read the csv
        df = pd.read_csv(ddos_files[i], header=None, skiprows=5, names=protocol_names)
        
        # Drop rows we don't need, including those where the src IP is the victim
        df = df.drop(df.tail(1).index)
        # TODO: Change so that pairs where victimIP is src is labelled
        # victim_response instead of being dropped.
        df = df[df[protocol + "_src"] != victimIP]
        df = df.drop(protocol_to_drop, axis=1)
        
        protocol_names.remove(protocol + "_src")
        protocol_names.remove(protocol + "_<->")
        protocol_names.remove(protocol + "_dest")
        protocol_names.remove(protocol + "_in_frames")
        protocol_names.remove(protocol + "_in_bytes")
        protocol_names.remove(protocol + "_total_frames")
        protocol_names.remove(protocol + "_total_bytes")
        # Now add information in df to ddos_df
        for protocol_name in protocol_names:
            ddos_df[protocol_name] = 0
        
        for row_index, row in df.iterrows():
            protocol_source_ip = row[protocol + '_src']
            idx = ddos_df.loc[ddos_df['total_src'] == protocol_source_ip].index
            idx_int = idx[0]
            
            for protocol_name in protocol_names:
                ddos_df.loc[idx_int, protocol_name] = row[protocol_name]
#             for protocol_name in protocol_names:
#                 print("PROTOC NAME: {0}".format(protocol_name))
#                 print("COLUMN:      {0}".format('total' + protocol_name[len(protocol):]))
#                 print("DDOS_DF[PR]: {0}".format(ddos_df.loc[idx_int]))
#                 print("ROW[PROTOC]: {0}".format(row[protocol_name]))                            
                #                 ddos_df.loc[idx[0], 'total' + protocol_name[len(protocol):]] = row[protocol_name]
#                 print(ddos_df.loc[idx[0], 'total' + protocol_name[len(protocol):]])

PROTOCOL : icmp
PROTOCOL : total


In [38]:
x = ddos_df.loc[ddos_df['total_src'] == "195.52.240.138"].index
ddos_df.loc[x, 'src'] = 34
print(ddos_df.loc[x, 'src'])
print(x)

2    34
Name: src, dtype: int64
Int64Index([2], dtype='int64')


In [188]:
# Want to match the source IP in the current csv to that in the ddos_df
ddos_row_index = ddos_df.loc[ddos_df['total_src'] == "192.253.3.155"].index
print(ddos_row_index)
print(ddos_row_index[0])
# print(ddos_row_index)
# print(ddos_df.loc[226])
# print(ddos_df.loc[226, 'total_out_bytes'])
# ddos_df.loc[226, 'total_out_bytes'] = 2000
# print(ddos_df.loc[226, 'total_out_bytes'])

Int64Index([226], dtype='int64')
226


In [13]:
# Verifying that all src IPs are distinct...
def anydup(thelist):
  seen = set()
  for x in thelist:
    if x in seen: return True
    seen.add(x)
  return False

print(anydup(ddos_df.total_src))

False


In [18]:
type(ddos_df["total_src"])

pandas.core.series.Series

In [190]:
ddos_df.h

,total_src,total_dest,total_in_frames,total_in_bytes,total_out_frames,total_out_bytes,total_rel_start,total_duration,icmp_out_frames,icmp_out_bytes,icmp_rel_start,icmp_duration
1,195.216.1.86,71.126.222.64,1,76,37,2220,0.035257,0.4714,0,0,0,0
2,195.52.240.138,71.126.222.64,1,76,33,1980,0.038455,0.4708,0,0,0,0
4,209.210.127.200,71.126.222.64,0,0,49,2940,0.007926,0.4152,0,0,0,0
5,167.46.3.209,71.126.222.64,1,88,27,1620,0.000971,0.5058,0,0,0,0
6,217.238.32.123,71.126.222.64,1,76,26,1560,0.028566,0.4766,0,0,0,0
7,192.229.242.166,71.126.222.64,1,88,27,1620,0.009343,0.4752,0,0,0,0
8,130.119.190.246,71.126.222.64,0,0,25,1500,0.006991,0.5050,0,0,0,0
11,193.102.212.129,71.126.222.64,0,0,51,3060,0.009274,0.4891,0,0,0,0
12,195.38.237.153,71.126.222.64,0,0,19,1140,0.062032,0.4424,0,0,0,0
13,193.92.189.75,71.126.222.64,0,0,49,2940,0.008604,0.5023,0,0,0,0
